# IR EDA

## 1. documents.isonl

In [22]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
# JSONL 파일 읽기
data = []
with open('/root/documents.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# DataFrame으로 변환
df = pd.DataFrame(data)

In [14]:
# 0. 데이터 샘플 추출

print("처음 5개 행:")
print(df.head())

처음 5개 행:
                                  docid  \
0  42508ee0-c543-4338-878e-d98c6babee66   
1  4a437e7f-16c1-4c62-96b9-f173d44f4339   
2  d3c68be5-9cb1-4d6e-ba18-5f81cf89affb   
3  910107a6-2a42-41a2-b337-fbf22d6440fe   
4  74f22819-1a8e-4646-8a9d-13323de8cdb8   

                                     src  \
0               ko_mmlu__nutrition__test   
1      ko_mmlu__conceptual_physics__test   
2        ko_ai2_arc__ARC_Challenge__test   
3        ko_ai2_arc__ARC_Challenge__test   
4  ko_ai2_arc__ARC_Challenge__validation   

                                             content  content_length  
0  건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 ...             381  
1  수소, 산소, 질소 가스의 혼합물에서 평균 속도가 가장 빠른 분자는 수소입니다. 수...             231  
2  종이와 플라스틱은 재활용 가능한 자원입니다. 중학교 과학 수업에서 우리는 종이와 플...             517  
3  마이애미파랑나비는 남부 플로리다에서 멸종 위기에 처한 종입니다. 이 나비의 개체수 ...             371  
4  비버는 나무를 베고, 덤불과 관목을 모아 강과 개울에 댐을 만드는 것으로 알려져 있...             503  


#### => 이렇게 doc_id/src/content로 나눠져있고, 각각 문서별 id/출처/지식정보로 예상됨

In [ ]:
# 1. 기본 정보 확인 
print("데이터셋 크기:", len(df))
print("\n소스(src) 별 문서 수:")
print(df['src'].value_counts())

데이터셋 크기: 4272

소스(src) 별 문서 수:
src
ko_ai2_arc__ARC_Challenge__test          943
ko_ai2_arc__ARC_Challenge__train         866
ko_ai2_arc__ARC_Challenge__validation    238
ko_mmlu__conceptual_physics__test        211
ko_mmlu__nutrition__test                 168
                                        ... 
ko_mmlu__high_school_physics__train        1
ko_mmlu__college_chemistry__train          1
ko_mmlu__nutrition__train                  1
ko_mmlu__high_school_chemistry__train      1
ko_mmlu__high_school_biology__train        1
Name: count, Length: 63, dtype: int64


#### => 고등학교 내용, 간호학, 화학, 물리학 등에서 가져온 내용으로 보임 

In [3]:
# 2. 문서 길이 분석
df['content_length'] = df['content'].str.len()
print("\n문서 길이 통계:")
print(df['content_length'].describe())


문서 길이 통계:
count    4272.000000
mean      315.326779
std       103.983172
min        44.000000
25%       251.000000
50%       299.000000
75%       357.000000
max      1230.000000
Name: content_length, dtype: float64


In [ ]:
# 3. 위에서 결과로 나온 min/50%/max 에 대한 문서 길이별 예시 추출

# 가장 짧은 문서
shortest_doc = df.loc[df['content_length'].idxmin()]
print("=== 가장 짧은 문서 (44자) ===")
print(f"문서 ID: {shortest_doc['docid']}")
print(f"소스: {shortest_doc['src']}")
print(f"길이: {shortest_doc['content_length']}")
print("내용:")
print(shortest_doc['content'])
print("\n" + "="*80 + "\n")

# 중간 길이의 문서 (중앙값에 가장 가까운 문서)
median_length = df['content_length'].median()
middle_doc = df.iloc[(df['content_length'] - median_length).abs().idxmin()]
print("=== 중간 길이 문서 (약 299자) ===")
print(f"문서 ID: {middle_doc['docid']}")
print(f"소스: {middle_doc['src']}")
print(f"길이: {middle_doc['content_length']}")
print("내용:")
print(middle_doc['content'])
print("\n" + "="*80 + "\n")

# 가장 긴 문서
longest_doc = df.loc[df['content_length'].idxmax()]
print("=== 가장 긴 문서 (1,230자) ===")
print(f"문서 ID: {longest_doc['docid']}")
print(f"소스: {longest_doc['src']}")
print(f"길이: {longest_doc['content_length']}")
print("내용:")
print(longest_doc['content'])

=== 가장 짧은 문서 (44자) ===
문서 ID: d9760a71-24e5-4896-9ecd-82a5628de792
소스: ko_mmlu__nutrition__validation
길이: 44
내용:
북유럽과 남유럽 국가들에서 상당히 유사한 섭취를 보이는 영양소는 PUFA입니다.


=== 중간 길이 문서 (약 299자) ===
문서 ID: 25e5f74a-d387-43b8-891d-6f55de5dc4bc
소스: ko_mmlu__virology__test
길이: 299
내용:
RNA 바이러스는 과도하게 변이하는 바이러스입니다. 이러한 바이러스는 RNA를 유전 정보로 사용하며, 그 변이 속도는 매우 빠릅니다. RNA 바이러스는 유전자 변이를 통해 새로운 변종을 만들어내고, 이는 종종 새로운 질병의 원인이 됩니다. 이러한 변이는 바이러스의 생존 전략 중 하나로, 환경 변화에 대응하기 위해 진화하는 과정입니다. RNA 바이러스의 과도한 변이는 백신 개발과 치료 방법의 어려움을 야기할 수 있으며, 전염성과 병원성을 증가시킬 수도 있습니다. 따라서 RNA 바이러스의 변이에 대한 연구와 감시는 중요한 과제입니다.


=== 가장 긴 문서 (1,230자) ===
문서 ID: 1a58b722-ce6c-4cf6-9285-9336c2967301
소스: ko_mmlu__college_physics__test
길이: 1230
내용:
고체 디스크가 휴식 상태에서 시작하여 경사면을 따라 내려간다는 상황을 가정해보겠습니다. 이 디스크는 균일한 구조를 가지고 있으며, 미끄러지지 않고 경사면을 따라 움직입니다.

일정 시간이 지난 후, 디스크의 총 운동 에너지 중 회전 운동 에너지가 차지하는 비율을 알아보고자 합니다. 회전 운동 에너지는 디스크의 회전에 의해 발생하는 에너지로, 디스크의 질량과 반지름에 의해 결정됩니다.

이 문제에서는 디스크가 휴식 상태에서 시작하므로, 디스크의 운동 에너지는 전적으로 회전 운동 에너지로 이루어져 있습니다. 따라서 디스크의 총 운동 에너지 중 회전 운동 

In [5]:
# 4. 자주 등장하는 키워드 분석
def get_keywords(text):
    # 간단한 키워드 추출 (공백 기준 분리)
    return [word for word in text.split() if len(word) > 1]

keywords = []
for content in df['content']:
    keywords.extend(get_keywords(content))

print("\n상위 20개 키워드:")
print(Counter(keywords).most_common(20))



상위 20개 키워드:
[('있습니다.', 5927), ('이러한', 3471), ('이는', 2337), ('합니다.', 2335), ('따라서,', 2240), ('중요한', 2111), ('역할을', 1769), ('통해', 1650), ('가장', 1511), ('있는', 1348), ('다양한', 1307), ('영향을', 1262), ('있으며,', 1244), ('위해', 1207), ('가지고', 1159), ('인해', 1148), ('따라서', 1119), ('다른', 1037), ('대한', 990), ('이를', 969)]


In [6]:
# 5. docid 형식 확인
print("\ndocid 형식 샘플:")
print(df['docid'].head())


docid 형식 샘플:
0    42508ee0-c543-4338-878e-d98c6babee66
1    4a437e7f-16c1-4c62-96b9-f173d44f4339
2    d3c68be5-9cb1-4d6e-ba18-5f81cf89affb
3    910107a6-2a42-41a2-b337-fbf22d6440fe
4    74f22819-1a8e-4646-8a9d-13323de8cdb8
Name: docid, dtype: object


In [ ]:
# 6. 중복 문서 확인 
print("\n중복 문서 수:")
print(df['content'].duplicated().sum())


중복 문서 수:
6


#### => 실제로 확인해보니 중복되지 않아서 아래 완전 중복된 내용 다시 추출 > 12개 있음

In [ ]:
# 7. 다시 완전히 동일한 content를 가진 문서들 찾기 
duplicate_groups = df[df.duplicated(subset=['content'], keep=False)].sort_values('content')

print("완전히 동일한 내용을 가진 문서 그룹:")
for content in duplicate_groups['content'].unique():
    same_content = df[df['content'] == content]
    if len(same_content) > 1:  # 2개 이상인 경우만 출력
        print("\n=== 동일 내용 문서 그룹 ===")
        for idx, row in same_content.iterrows():
            print(f"\n문서 ID: {row['docid']}")
            print(f"소스: {row['src']}")
            print("내용:")
            print(row['content'])
        print("="*80)

완전히 동일한 내용을 가진 문서 그룹:

=== 동일 내용 문서 그룹 ===

문서 ID: d758612c-6dfb-47b4-991d-6f053b282264
소스: ko_mmlu__high_school_physics__test
내용:
공기에서 유리창 판으로 들어오는 단색광 빔은 속도와 파장에 변화가 일어납니다. 유리창을 통과하는 빛은 공기와 유리의 굴절률 차이로 인해 속도가 변하게 됩니다. 이로 인해 빛의 파장도 변화하게 되는데, 이를 파장 변화라고 합니다. 공기에서 유리창으로 들어오는 단색광 빔은 이러한 변화를 겪게 됩니다.

문서 ID: 318f4fb1-92d2-4b4c-914f-f9ff8b885aed
소스: ko_mmlu__high_school_physics__test
내용:
공기에서 유리창 판으로 들어오는 단색광 빔은 속도와 파장에 변화가 일어납니다. 유리창을 통과하는 빛은 공기와 유리의 굴절률 차이로 인해 속도가 변하게 됩니다. 이로 인해 빛의 파장도 변화하게 되는데, 이를 파장 변화라고 합니다. 공기에서 유리창으로 들어오는 단색광 빔은 이러한 변화를 겪게 됩니다.

=== 동일 내용 문서 그룹 ===

문서 ID: d2145d8e-45aa-4ada-9b82-db197e7ee39a
소스: ko_mmlu__college_physics__train
내용:
굴절 망원경은 굴절 렌즈로 구성되어 있으며, 이는 100 cm로 분리된 두 개의 수렴 렌즈로 이루어져 있습니다. 망원경의 안경 렌즈의 초점 거리는 20 cm입니다. 이러한 조건에서 망원경의 각배율은 4입니다. 각배율은 망원경의 능력을 나타내는 지표로, 망원경으로 관찰하는 대상의 크기를 실제 크기에 비해 얼마나 크게 보여주는지를 나타냅니다. 따라서, 이 굴절 망원경은 관찰 대상을 4배로 확대하여 보여줄 수 있습니다.

문서 ID: b3da4d1b-f001-4632-92ab-bcc6bc9c2cc1
소스: ko_mmlu__college_physics__train
내용:
굴절 망원경은 굴절 렌즈로 구성되어 있으

## 2. eval.jsonl

#### 총 220개 중 20개는 일반 대화 20개는 멀티턴 대화로 구성

In [26]:
# JSONL 파일 읽기
data = []
with open('/root/eval.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [34]:
# 1. 기본 데이터 정보
print("총 대화 수:", len(df))
print("\n데이터프레임 기본 정보:")
print(df.info())

총 대화 수: 220

데이터프레임 기본 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   eval_id        220 non-null    int64 
 1   msg            220 non-null    object
 2   dialogue_type  220 non-null    object
 3   msg_length     220 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 7.0+ KB
None


In [35]:
# 2. msg 구조 분석
# msg 필드의 첫 번째 메시지 내용 추출
df['user_content'] = df['msg'].apply(lambda x: x[0]['content'])

In [36]:
# 3. 메시지 길이 분석
df['content_length'] = df['user_content'].str.len()
print("\n메시지 길이 통계:")
print(df['content_length'].describe())


메시지 길이 통계:
count    220.000000
mean      24.236364
std        9.610617
min        6.000000
25%       18.000000
50%       23.000000
75%       29.000000
max       70.000000
Name: content_length, dtype: float64


In [38]:
# 4. 가장 긴/짧은 질문 5개씩 확인
print("\n=== 가장 긴 질문 5개 ===")
longest_msgs = df.nlargest(5, 'content_length')
for idx, row in longest_msgs.iterrows():
    print(f"\neval_id: {row['eval_id']}")
    print(f"길이: {row['content_length']}")
    print("질문내용:")
    print(row['user_content'])
    print("-"*80)
    
print("\n=== 가장 짧은 질문 5개 ===")
shortest_msgs = df.nsmallest(5, 'content_length')
for idx, row in shortest_msgs.iterrows():
    print(f"\neval_id: {row['eval_id']}")
    print(f"길이: {row['content_length']}")
    print("질문내용:")
    print(row['user_content'])
    print("-"*80)


=== 가장 긴 질문 5개 ===

eval_id: 277
길이: 70
질문내용:
바닥의 마찰이 없는 곳에서 두 사람이 서로 밀어낼때 무게가 덜 나가는 사람이 더 나가는 사람이 더 빨리 움직이게 되는 이유는?
--------------------------------------------------------------------------------

eval_id: 243
길이: 55
질문내용:
사람이나 물체가 지구 위에서 땅속으로 꺼지거나 바깥으로 튕겨나가지 않고 가만히 서 있을 수 있잖아?
--------------------------------------------------------------------------------

eval_id: 241
길이: 51
질문내용:
정육면체가 가라앉지 않고 물 위에 떠 있을 때 수면 윗부분에 해당하는 부피를 구하는 방법은?
--------------------------------------------------------------------------------

eval_id: 293
길이: 49
질문내용:
두 물질이 다른 분자구조나 화학적인 성분으로 이루어져 있다는 것을 어떻게 알 수 있나요?
--------------------------------------------------------------------------------

eval_id: 37
길이: 48
질문내용:
두개의 소스로부터 발생한 사건중 어떤 쪽에서 기인한 것인지 확률 계산하는 예시 알려줘.
--------------------------------------------------------------------------------

=== 가장 짧은 질문 5개 ===

eval_id: 90
길이: 6
질문내용:
안녕 반갑다
--------------------------------------------------------------------------------

eval_id: 222
길이: 6
질문내용:
안녕 

In [39]:
# 5. 랜덤 샘플 5개 확인
print("\n=== 랜덤 질문 샘플 5개 ===")
random_samples = df.sample(n=5)
for idx, row in random_samples.iterrows():
    print(f"\neval_id: {row['eval_id']}")
    print(f"길이: {row['content_length']}")
    print("질문내용:")
    print(row['user_content'])
    print("-"*80)


=== 랜덤 질문 샘플 5개 ===

eval_id: 26
길이: 21
질문내용:
짚신 벌레의 번식은 어떻게 이루어지나?
--------------------------------------------------------------------------------

eval_id: 33
길이: 13
질문내용:
Python 공부중이야.
--------------------------------------------------------------------------------

eval_id: 215
길이: 27
질문내용:
디엔에이와 단백질의 관계와 역할에 대해 설명해줘.
--------------------------------------------------------------------------------

eval_id: 42
길이: 13
질문내용:
이란 콘트라 사건이 뭐야
--------------------------------------------------------------------------------

eval_id: 77
길이: 43
질문내용:
새로운 생명체의 발생에 대한 오래된 믿음을 깨기 위해 어떤 방법이 사용되었나?
--------------------------------------------------------------------------------


In [40]:
# 6. 질문 길이 구간별 분포
print("\n질문 길이 구간별 분포:")
length_bins = [0, 10, 20, 30, 40, 50, float('inf')]
df['length_category'] = pd.cut(df['content_length'], bins=length_bins)
print(df['length_category'].value_counts().sort_index())


질문 길이 구간별 분포:
length_category
(0.0, 10.0]     11
(10.0, 20.0]    67
(20.0, 30.0]    96
(30.0, 40.0]    33
(40.0, 50.0]    10
(50.0, inf]      3
Name: count, dtype: int64


In [43]:
# 6-1. 가장 적은 구간의 샘플
distribution = df['length_category'].value_counts().sort_index()

min_category = distribution.idxmin()
print(f"\n=== 가장 적은 구간 ({min_category})의 샘플 ===")
min_samples = df[df['length_category'] == min_category]
for idx, row in min_samples.iterrows():
   print(f"\neval_id: {row['eval_id']}")
   print(f"길이: {row['content_length']}")
   print("질문내용:")
   print(row['user_content'])
   print("-"*80)

# 6-2. 가장 많은 구간의 샘플
max_category = distribution.idxmax()
print(f"\n=== 가장 많은 구간 ({max_category})의 샘플 5개 ===")
max_samples = df[df['length_category'] == max_category].sample(n=5)
for idx, row in max_samples.iterrows():
   print(f"\neval_id: {row['eval_id']}")
   print(f"길이: {row['content_length']}")
   print("질문내용:")
   print(row['user_content'])
   print("-"*80)


=== 가장 적은 구간 ((50.0, inf])의 샘플 ===

eval_id: 241
길이: 51
질문내용:
정육면체가 가라앉지 않고 물 위에 떠 있을 때 수면 윗부분에 해당하는 부피를 구하는 방법은?
--------------------------------------------------------------------------------

eval_id: 243
길이: 55
질문내용:
사람이나 물체가 지구 위에서 땅속으로 꺼지거나 바깥으로 튕겨나가지 않고 가만히 서 있을 수 있잖아?
--------------------------------------------------------------------------------

eval_id: 277
길이: 70
질문내용:
바닥의 마찰이 없는 곳에서 두 사람이 서로 밀어낼때 무게가 덜 나가는 사람이 더 나가는 사람이 더 빨리 움직이게 되는 이유는?
--------------------------------------------------------------------------------

=== 가장 많은 구간 ((20.0, 30.0])의 샘플 5개 ===

eval_id: 258
길이: 30
질문내용:
머클-담고르 해시 함수를 사용한 MAC 보안 취약점은?
--------------------------------------------------------------------------------

eval_id: 306
길이: 24
질문내용:
은하에는 엄청나게 많은 별들이 모여 있잖아?
--------------------------------------------------------------------------------

eval_id: 255
길이: 28
질문내용:
어떤 물체를 매우 크게 확대해서 볼수 있는 기계는?
--------------------------------------------------------------------------------

eval_id: 45

In [41]:
# 7. 자주 등장하는 키워드 분석
def get_keywords(text):
    return [word for word in text.split() if len(word) > 1]

keywords = []
for content in df['user_content']:
    keywords.extend(get_keywords(content))

print("\n상위 20개 키워드:")
print(Counter(keywords).most_common(20))


상위 20개 키워드:
[('대해', 49), ('알려줘.', 33), ('어떻게', 19), ('설명해줘.', 16), ('어떤', 14), ('있는', 14), ('있어?', 13), ('방법은?', 11), ('이유는?', 11), ('뭐야?', 11), ('미치는', 9), ('영향은?', 7), ('하는', 7), ('역할에', 7), ('너무', 6), ('뭐가', 6), ('위한', 5), ('있나?', 5), ('원리는?', 5), ('가장', 5)]


In [42]:
# 8. 질문 끝나는 형태 분석
print("\n질문 종결 형태:")
endings = df['user_content'].apply(lambda x: x[-1] if len(x) > 0 else '')
print(endings.value_counts().head())


질문 종결 형태:
user_content
?    144
.     64
!      7
야      1
구      1
Name: count, dtype: int64
